In [1]:
# latest update
# April 26, 2017
# Changes done by: Jimmy Vivas
# Added code description

# This code was developed by Jimmy Vivas

# The code will take all CSv files from a specific directory and process them at the same Time
# The code will get replace strings in the values for numeric data
#   It will assign an integer value for each distinct string for each column (sensor)
#   The integer value will be reset it for each sensor
#   The code does not evaluare wether or not the sensor should stay. THe analyst will make this decision
#   Using Previse or any other tool
# The code will split the csv file if the result is more than 1MM rows

In [2]:
from os import listdir
import pandas as pd
import math
import numpy as np

In [3]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

# CSVPath1 = '/Users/jvivas/Documents/Aspen/GASCO/Sensor Data/To Be Processed'

# Windows Path
CSVPath1 = 'C:/Users/vivasj/Documents/Aspen/PTT GC/C002B/DATA/To be Processed'
FinalPath = 'C:/Users/vivasj/Documents/Aspen/PTT GC/C002B/DATA/Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [4]:
CSVFileList = []
NanPHDTagList = []
CSVFileListAll = listdir(CSVPath1)

m = len(CSVFileListAll)

for i in range(m):
    fileNameStr = CSVFileListAll[i]
    fileStr = fileNameStr.split('.')[0]
    fileExt = fileNameStr.split('.')[1]
    if fileExt == "csv":
        CSVFileList.append(fileNameStr)

n = len(CSVFileList)

# Uncomment following cell for debugging purposes

In [5]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# np.set_printoptions(threshold=np.nan)

In [6]:
def GetFileSize(file):
    import os
    csvFileStatInfo = os.stat(file)
    csvFileSizeGB = csvFileStatInfo.st_size/1000000000
    return(csvFileSizeGB)    

In [7]:
def RenameColumn(df2):
# Ranane Date and time column to DATETIME
    new_cols = ['DATETIME']
    df2.rename(columns=dict(zip(df2.columns[[0]], new_cols)),inplace=True)
    return

In [8]:
def ApplyDateFormat(df2):

    # duplicate df2 to apply date and time format 
    # df2 = df2.copy()    
    # Change new datetime column to datetime format
    df2['DATETIME'] = pd.to_datetime(df2['DATETIME'])    
    # Change datetime column format to look like 01/31/2015 0:00:00
    df2['DATETIME'] = df2['DATETIME'].dt.strftime('%m/%d/%Y %H:%M:%S')
    return

In [9]:
def SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile):
    chunksize = 25000

    i = 0
    j = 1
    
    SensorStringListAllChunks = []
    StringListForAllSensors = []
    StringListForEachChunk = []
        
    print ('Loading ' + CSVFile + ' file')
    for df in pd.read_csv(CSVFileWithPath, chunksize=chunksize, iterator=True, low_memory=False):
        # df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
        df.index += j
        i+=1
        j = df.index[-1] + 1
        h = 0
        
        # Rename date and time column
        print ('Renaming date and time column')
        RenameColumn(df)

        # Apply Date and time format to dataframe
        print ('Applying date and time format')
        ApplyDateFormat(df)

        # Code to do Transposing
        # Create two dataframes df1 only with tags and descriptions. df2 tag with values
        if i == 1:
            df1 = df[0:1]  # FIRST ROW
            # Indexing dataframe df1
            df1 = df1.set_index('DATETIME')
        else:
            if i > 1:
                df1 = df1
                
        df2 = df[1:len(df)]  # SECOND TO LAST ROW

        # Indexing dataframe df2
        df2 = df2.set_index('DATETIME')
        
        # Export CSV chunk for each loop
        print ('Exporting chunk' + str(i))
        df.to_csv(CSVFileWithPath.replace('.csv', '') + '_chunk_ ' + str(i) + '.csv', index=True)
        
        print ('Getting the list for chunk' + str(i))
        StringListForEachChunk = ExtractStrings(df1, df2)
        
        print ('These are all the strings found in the chunk ' + str(i) + ':' + str(StringListForEachChunk))
        
        SensorStringListAllChunks.extend(StringListForEachChunk)

    SensorStringListAllChunks = list(set(SensorStringListAllChunks))
    return(SensorStringListAllChunks)

In [10]:
def ExtractStrings(df2_1, df2_2):
        
    print ('Converting df to numeric')
    # Convert columns to numbers those that has string wil be converted to numpy null = NaN
    df2_with_nan = df2_2.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # print (df2_2.head())
    # print (df2_with_nan.head())
    # Ge the list of sensors with Strings
    df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))
    df3 = df3.reset_index()
    SensorsWithStrings = df3[df3[0]==True]['index'].tolist()
    
    # Replace nan with 'null' and create new df
    df2_with_null = df2_with_nan.fillna(value='null')
    
    # Here we filter the columns that are objects and create a dataframe with those columns
    colsObject = df2_2.select_dtypes(include=['object']).columns
    TotalColumns = df2_2.columns
    
    print ('Total columns with text: ' + str(TotalColumns))
    print ('Sensors with text in their Values: ' + str(SensorsWithStrings))

    AllSensorsStringList = []
    
    if len(SensorsWithStrings) == 0:
        print ('No strings in this dataframe')
    else:
        # print (str(len(SensorsWithStrings)) + ' columns contain strings out of ' + str(len(TotalColumns)) + ' columns')
        # if (len(SensorsWithStrings)/len(TotalColumns)) > 0.5:
            # print ('this process will take some time depending of the size of the file and the PC resources')

        # Dropping DATETIME index to merge df1 and df2
        df2_1 = df2_1.reset_index(drop=False)
        df2_2 = df2_2.reset_index(drop=False)
        
        # Variable initialization
        j = 0

        StringListForCurrentSensor = []

        # Create dictionary from dataframe columns (sensors) that have strings only
        SensorDictionary = {}.fromkeys(SensorsWithStrings, [])
        
        # Loop to go thru each column and convert the characters string to numbers    
        for j in range(len(SensorsWithStrings)):
            # from IPython.core.debugger import Tracer
            # Tracer()() #this one triggers the debugger

            # iterate thru each column in the dataframe
            # for j in range(len(list(SensorDictionary))):
            # update sensor name for each column
            Sensor = list(SensorDictionary)[j]

            print ('Processing Sensor: ' + Sensor)
            # Clear List of String for each Sensor
            SensorStringResult = []

            # Get rows that have null in the actual sensor column
            result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

            # Convert list of nulls to dataframe and reset the datetime index
            result_df = pd.DataFrame(result)
            result_df = result_df.reset_index()
            
            # print ('printing result_df')
            # print (result_df.head())
            # print ('printing df2_2')
            # print (df2_2.head())
            # debugging
            # print (result_df)
            # debugging

            # Filtering rows with String for each sensor
            SensorStringResult = df2_2[df2_2['DATETIME'].isin(result_df.loc[:,"DATETIME"].values.tolist())][Sensor]

            # Adding Strings to Sensor in Sensor Dictionary
            # I think that when due to a bug the dictionary become too big it stops growing...
            # SensorDictionary[Sensor] = list(set(SensorStringResult))
            StringListForCurrentSensor = list(set(SensorStringResult))
            
            # Debugging
            # print (list(set(SensorStringResult)))
            # Debugging
            
            # print ('Sensor: ' + Sensor + ' Processed')

            # Adding String for Sensor to General String List
            AllSensorsStringList.extend(StringListForCurrentSensor)

            # Removing Duplicate Strings
            AllSensorsStringList = list(set(AllSensorsStringList))
            
            print ('These are all the strings found in Sensor ' + Sensor + ': ' + str(StringListForCurrentSensor))
    return (AllSensorsStringList)

In [11]:
def SetIndex(df2):
        # Code to do Transposing
    # Create two dataframes df1 only with tags and descriptions. df2 tag with values
    df2_1 = df2[0:1]  # FIRST ROW
    df2_2 = df2[1:len(df2)]  # SECOND TO LAST ROW
    
    # Indexing dataframe df1
    df2_1 = df2_1.set_index('DATETIME')

    # Indexing dataframe df2
    df2_2 = df2_2.set_index('DATETIME')
    return(df2_1, df2_2)

In [12]:
def LoadCSV(FileAndPath, file):
    print ('Loading ' + file)
    df2 = pd.read_csv(FileAndPath, low_memory=False)
    return(df2)

In [13]:
def FormatToPrevise(df2_1, df2_2):
    # Dropping DATETime index to merge df1 and df2
    df2_1 = df2_1.reset_index(drop=False)
    df2_2 = df2_2.reset_index(drop=False)

    # Converting Historian files to VTQ format (DATETime, TAGNAME, DESCRIPTION, VALUE)
    mdf = pd.merge(pd.melt(df2_1, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='DESCRIPTION')[['TAGNAME', 'DESCRIPTION']],
                   pd.melt(df2_2, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='VALUE'),
                   on=['TAGNAME'])

    # Sort columns by VTQ format
    mdf = mdf[['DATETIME', 'TAGNAME', 'DESCRIPTION', 'VALUE']]
    
    return (mdf)

In [14]:
def SplitPreviseFormatCSVFile(mdf):
    # Exporting PHD Tag CSV file
    i = 0
    rows = 1000000
    totalRows = len(mdf)
    loops = math.ceil(totalRows/rows) + 1

    if totalRows > 1000000:
        for j in range(loops): #need to round this
            j = j + 1
            print (str(j))
            a = (rows*j) - rows
            if totalRows <= rows:
                b = totalRows
                print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' Historian File')
                print("")
                mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted.csv', index=False)
            else:
                if (rows*j) >= totalRows:
                    b = totalRows
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
                else:
                    b = (rows*j) - 1
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
    else:
        mdf.to_csv(FinalPath + '/' + str(CSVFileList[0].replace('.csv', '')) + '_Formatted' + '.csv', index=False)
    return

# This is the Main Code to execute

In [ ]:
CSVFile = CSVFileList[0]
CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

csvFileSizeGB = GetFileSize(CSVFileWithPath)

df2 = LoadCSV(CSVFileWithPath, CSVFile)

In [19]:
df2_with_nan = df2_2.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [21]:
df2_with_nan.head()

,A1-200TI727HA.PV,A1-200TI727HB.PV,A1-200XI952B.PV,A1-200TI726AB.PV,A1-200TI726CB.PV,A1-200TI726EB.PV,A1-200VI728DB.PV,A1-200VI728EB.PV,A1-200VI728FB.PV,A1-200TI727AB.PV,...,A1-200PI747BB.PV,A1-200PI747CB.PV,A1-200PI751B.PV,A1-200TI812.PV,A1-200PI814.PV,100TI97.PV,200PI53.PV,200TI371.PV,200TI348.PV,200TI347.PV
DATETIME,,,,,,,,,,,,,,,,,,,,,
01/01/2014 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.728256,2.518174,69.249283,60.885456,56.545738
01/01/2014 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.606651,2.508537,69.088890,60.658840,56.511757
01/01/2014 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.528124,2.502900,69.039642,60.466503,56.509052
01/01/2014 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.647432,2.497166,69.006332,60.593910,56.434353
01/01/2014 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.664402,2.491784,68.754303,60.740746,56.482708


In [22]:
df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))

In [23]:
df3 = df3.reset_index()

In [24]:
SensorsWithStrings = df3[df3[0]==True]['index'].tolist()

In [25]:
df2_with_null = df2_with_nan.fillna(value='null')

In [29]:
colsObject = df2_2.select_dtypes(include=['object']).columns

In [27]:
TotalColumns = df2_2.columns

In [28]:
AllSensorsStringList = []

In [30]:
df2_1 = df2_1.reset_index(drop=False)
df2_2 = df2_2.reset_index(drop=False)

In [31]:
StringListForCurrentSensor = []

# Create dictionary from dataframe columns (sensors) that have strings only
SensorDictionary = {}.fromkeys(SensorsWithStrings, [])


In [ ]:
SensorDictionary

In [ ]:
Sensor = list(SensorDictionary)[0]

In [33]:
Sensor = '905II259BI.PV'

In [34]:
df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

DATETIME
02/16/2014 05:20:00   NaN
04/07/2014 17:45:00   NaN
04/28/2014 09:55:00   NaN
04/28/2014 10:00:00   NaN
04/28/2014 10:05:00   NaN
04/28/2014 10:10:00   NaN
04/28/2014 10:15:00   NaN
04/28/2014 10:20:00   NaN
04/28/2014 10:25:00   NaN
04/28/2014 21:50:00   NaN
04/28/2014 21:55:00   NaN
04/28/2014 22:00:00   NaN
04/28/2014 22:05:00   NaN
04/28/2014 22:10:00   NaN
04/28/2014 22:15:00   NaN
04/28/2014 22:20:00   NaN
04/28/2014 22:25:00   NaN
04/28/2014 22:30:00   NaN
04/28/2014 22:35:00   NaN
04/28/2014 22:40:00   NaN
04/28/2014 22:45:00   NaN
04/28/2014 22:50:00   NaN
04/28/2014 22:55:00   NaN
04/28/2014 23:00:00   NaN
04/28/2014 23:05:00   NaN
04/28/2014 23:10:00   NaN
04/28/2014 23:15:00   NaN
04/28/2014 23:20:00   NaN
04/28/2014 23:25:00   NaN
04/28/2014 23:30:00   NaN
                       ..
04/28/2015 00:10:00   NaN
04/28/2015 00:15:00   NaN
04/28/2015 00:20:00   NaN
04/28/2015 00:25:00   NaN
04/28/2015 00:30:00   NaN
06/13/2015 11:55:00   NaN
06/13/2015 12:00:00   NaN
06/

In [35]:
result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

In [36]:
result_df = pd.DataFrame(result)

In [37]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10216 entries, 02/16/2014 05:20:00 to 08/15/2016 21:35:00
Data columns (total 1 columns):
905II259BI.PV    0 non-null float64
dtypes: float64(1)
memory usage: 159.6+ KB


In [38]:
result_df = result_df.reset_index()

In [39]:
result_df.tail()

,DATETIME,905II259BI.PV
10211,08/15/2016 20:40:00,NaN
10212,08/15/2016 20:45:00,NaN
10213,08/15/2016 20:50:00,NaN
10214,08/15/2016 21:05:00,NaN
10215,08/15/2016 21:35:00,NaN


In [ ]:
result_df['DATETIME'].to_csv(FinalPath + '/result_df.csv',index=False)

In [52]:
Sensor

'905II259BI.PV'

In [43]:
df2_2[df2_2['DATETIME']=='08/15/2016 21:35:00'][Sensor]

271551    Bad
Name: 905II259BI.PV, dtype: object

In [ ]:
df2_2.loc[:, Sensor]
# df2.loc["Alaska":"Arkansas","2005":"2007"]

In [40]:
thelist = result_df.loc[:,'DATETIME'].values.tolist()

In [ ]:
df2_2[df2_2['DATETIME'].isin(result_df.loc[:,'DATETIME'].values.tolist())][Sensor]

In [15]:
CSVFile = CSVFileList[0]
CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

csvFileSizeGB = GetFileSize(CSVFileWithPath)

if csvFileSizeGB > 1:
    StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)
    print ('These are all the strings found in the file: ' + str(StringListForAllSensors))
else:
    
    df2 = LoadCSV(CSVFileWithPath, CSVFile)
    
    # Rename date and time column
    print ('Renaming date and time column')
    RenameColumn(df2)
    # Apply Date and time format to dataframe
    print ('Applying date and time format')
    ApplyDateFormat(df2)
    
    # Apply Index and create two dataframes
    df2_1, df2_2 = SetIndex(df2)
    
    print ('Extracting strings from csv file')
    StringListForAllSensors = ExtractStrings(df2_1, df2_2)
    
    print ('These are all the strings found in the csv file: ' + str(StringListForAllSensors))

# Convert all strings to a Dictionary
StringListDict = {}.fromkeys(StringListForAllSensors, 'null')

Loading 200C2_Fixed_Errors_JV.csv
Renaming date and time column
Applying date and time format
Extracting strings from csv file
Converting df to numeric
Total columns with text: Index(['A1-200TI727HA.PV', 'A1-200TI727HB.PV', 'A1-200XI952B.PV',
       'A1-200TI726AB.PV', 'A1-200TI726CB.PV', 'A1-200TI726EB.PV',
       'A1-200VI728DB.PV', 'A1-200VI728EB.PV', 'A1-200VI728FB.PV',
       'A1-200TI727AB.PV',
       ...
       'A1-200PI747BB.PV', 'A1-200PI747CB.PV', 'A1-200PI751B.PV',
       'A1-200TI812.PV', 'A1-200PI814.PV', '100TI97.PV', '200PI53.PV',
       '200TI371.PV', '200TI348.PV', '200TI347.PV'],
      dtype='object', length=120)
Sensors with text in their Values: ['A1-200TI727HA.PV', 'A1-200TI727HB.PV', 'A1-200XI952B.PV', 'A1-200TI726AB.PV', 'A1-200TI726CB.PV', 'A1-200TI726EB.PV', 'A1-200VI728DB.PV', 'A1-200VI728EB.PV', 'A1-200VI728FB.PV', 'A1-200TI727AB.PV', 'A1-200TI727BB.PV', 'A1-200TI727CB.PV', 'A1-200TI727DB.PV', 'A1-200TI727EB.PV', 'A1-200TI727FB.PV', 'A1-200TI727GB.PV', 'A1-20

These are all the strings found in Sensor A1-200XI949AB.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200XI949BB.PV
These are all the strings found in Sensor A1-200XI949BB.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200VI955BB.PV
These are all the strings found in Sensor A1-200VI955BB.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200TI745B.PV
These are all the strings found in Sensor A1-200TI745B.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200XI962B.PV
These are all the strings found in Sensor A1-200XI962B.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200XI951AB.PV
These are all the strings found in Sensor A1-200XI951AB.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200XI951BB.PV
These are all the strings found in Sensor A1-200XI951BB.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200VI957AB.PV
These are all the strings found in Sensor A1-

These are all the strings found in Sensor A1-200TI270B.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI271B.PV
These are all the strings found in Sensor A1-200TI271B.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: 200TI264.PV
These are all the strings found in Sensor 200TI264.PV: ['137.7579346', '137.5949402', '139.6115875', '139.4398804', '139.4484406', '138.0479279', '138.7314301', '35.25942993', '33.96323013', '139.4447174', '138.0436249', '137.7384949', '139.3518066', '139.4446564', '138.8851318', '138.2478943', '139.6477814', '139.7052765', '139.4508362', '139.1982422', '139.3701324', '138.6139832', '140.4440613', '34.73468781', '139.9050751', '140.266861', '139.3698578', '139.5689087', '139.8996429', '139.8816986', '33.67097473', '34.9084549', '137.9465179', '139.5924683', '139.4134979', '139.5716248', '138.3861542', '139.5564423', '138.5002747', '139.5788879', '140.2308655', '139.5651703', '139.5782318', '137.8867645', '139.2868958', '139.1874084', '138

These are all the strings found in Sensor A1-200PDI207.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200PDI208.PV
These are all the strings found in Sensor A1-200PDI208.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200PDI209.PV
These are all the strings found in Sensor A1-200PDI209.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: 200PIC48.PV
These are all the strings found in Sensor 200PIC48.PV: ['2.597833395', '2.601035118', '2.601588726', '2.60210371', '2.607871294', '2.600326777', '2.59609127', '2.600165129', '2.601446629', '2.600291729', '2.606811523', '2.59599328', '2.595217466', '2.590592623', '2.602257729', '2.608343124', '2.596297026', '2.601797581', '2.615571499', '2.597266912', '2.611618042', '2.602416277', '2.590088367', '2.595817089', '2.60353756', '2.57905364', '2.599059105', '2.60236764', '2.596321344', '2.600152254', '2.61861372', '2.604053736', '2.594440937', '2.612798452', '2.596668959', '2.597543955', '2.585790396', '2.605839252', '

These are all the strings found in Sensor 200LI46A.PV: ['0.000131298', '0.145118237', '0.024207445', '0.16543898', '1.130216718', '0.132700458', '0.170446485', '0.178026199', '0.211757988', '3.354283333', '0.121926717', '0.119403943', '0.137509316', '0.044250961', '0.067927286', '0.195552588', '0.110027932', '0.060995564', '0.091440126', '0.055678494', '0.061040163', '7.094518185', '0.082337767', '0.21911712', '0.063667983', '0.142768323', '-0.011356964', '0.033401076', '-0.013677696', '0.061017781', '0.151827797', '0.10205029', '0.137267187', '0.060940295', '0.04862516', '0.168964088', '0.142191768', '0.152702183', '0.20987317', '0.243326738', '0.091452427', '0.219557524', '0.122103527', '0.158654898', '0.177697554', '0.096074603', '0.24453418', '0.134131521', '0.210984647', '2.588922024', '0.122592367', '0.171347857', '0.123313151', '0.12836194', '0.110310338', '0.064669222', '0.265973687', '0.083037399', '0.205695316', '0.053335674', '0.091185138', '0.073788822', '0.174069777', '0.0

These are all the strings found in Sensor A1-200PI701B.PV: ['No Data', 'Error', 'Configure', 'Bad']
Processing Sensor: A1-200TI712B.PV
These are all the strings found in Sensor A1-200TI712B.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI722AB.PV
These are all the strings found in Sensor A1-200TI722AB.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI722BB.PV
These are all the strings found in Sensor A1-200TI722BB.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI722CB.PV
These are all the strings found in Sensor A1-200TI722CB.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI721AB.PV
These are all the strings found in Sensor A1-200TI721AB.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI721BB.PV
These are all the strings found in Sensor A1-200TI721BB.PV: ['No Data', 'Error', 'Configure']
Processing Sensor: A1-200TI721CB.PV
These are all the strings found in Sensor A1-200TI721CB.PV: ['No Data', 'Error', 'Conf

These are all the strings found in Sensor 200TI347.PV: ['60.83492661', '62.25540543', '61.55509949', '63.14767075', '59.85062027', '60.51099014', '59.74526215', '60.79807281', '61.96041489', '60.6024971', '61.19813538', '59.97549057', '59.15001297', '62.07512665', '58.4565506', '59.52326584', '62.23740387', '59.99806213', '60.9824791', '60.21697617', '60.36703491', '58.57011795', '63.33059692', '61.14933014', '61.42684555', '60.44485474', '61.57744217', '61.08337784', '59.69918442', '59.12236023', '63.18943024', '62.97017288', '58.99203873', '60.20560455', '62.83375168', '60.10144424', '60.2793808', '60.61169815', '61.42816544', '60.52009201', '60.37591171', '61.9167099', '59.8973732', '60.41622925', '61.4438591', '59.01161575', '63.20354462', '60.91725159', '60.11282349', '56.0567550701/09/2015 00:00', '60.3907814', '62.66120911', '59.86029816', '62.27516556', '64.34725189', '59.63465118', '63.11067581', '62.56613922', '59.52865982', '60.71999741', '61.15325546', '63.08061218', '59.21

In [ ]:
# CSVFile = CSVFileList[0]
# CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

# StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)

# Run following line to see the list of Strings
## Copy the result and paste it in the following line of code

In [ ]:
StringListDict

# Now we can replace string with null or a number
## I have assigned null to all strings by default all strings are replaced with null. If the strings need to be replaced with a number please replace the null with your desired number
###Example
###{'String 1': 'null',
    'String 2': 1,
    'String 3': 0}

### The replace code is still Work in Progress for files bigger than 1 GB
#### What to do
##### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the replace
##### 2. Once text is replaced the CSV file will be updated and load the next
##### 3. Once all csv file are processed We need to convert them to Previse Format

In [ ]:
# Replacing known test to null
df2_2.replace(
{'Bad': 0,
 'CLOSE': 0,
 'Comm Fail': 'null',
 'Configure': 'null',
 'Error': 'null',
 "Exception of type 'System.OutOfMemoryException' was thrown.": 'null',
 'Failed': 'null',
 'I/O Timeout': 'null',
 'Intf Shut': 'null',
 'MEM_ERR': 'null',
 'ON': 1,
 'OPEN': 1,
 'Off': 0,
 'RUN': 1,
 'STOP': 0,
 'Scan Off': 'null',
 'Scan Timeout': 'null'}
, inplace=True, regex=True)

# Now we convert the data to linear format Previse

## -----------------------------------------------------------------------------------------
## The format and Split will be done in the same step (this is to be done for files bigger than 1GB)
### The code will loop thru each chunk and will Format to Previse then will Split the file
### Each file will be a sensor
### If the sensor csv file has more than 1MM rows the code will split that file also
## -----------------------------------------------------------------------------------------

## The FormatToPrevise code is still Work in Progress for files bigger than 1 GB
### What to do
#### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the Formatting
#### 2. The code will loop thru each Splitted CSV file and format each one of them
#### 3. Once formatted the code will split the file into 1MM rows per CSV

In [ ]:
mdf = FormatToPrevise(df2_1,df2_2)

# Run following line to see the TOP 25 rows for the new Previse Format

In [ ]:
mdf.head(25)

In [ ]:
df2.info()

# Here we split the CSV data by file and each file will contain 1 million rows

### The SplitPreviseFormatCSVFile code is still Work in Progress for files bigger than 1 GB
### What to do

### Improvement
#### 1. I will update the code to create a CSV file per Sensor and if the sensor has more than 1MM rows it will splitted

In [ ]:
SplitPreviseFormatCSVFile(mdf)